# 1 상자로서의 변수

In [3]:
class Variable:
    def __init__(self, data):
        self.data = data

In [4]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [5]:
x = np.array(1)
x.ndim

0

In [6]:
x = np.array([1, 2, 3])
x.ndim

1

In [7]:
x = np.array([[1, 2, 3], [4, 5, 6]])
x.ndim

2

# 2 변수를 낳는 함수

In [8]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output

In [9]:
x = Variable(np.array([1,2,3,4]))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
[ 1  4  9 16]


In [10]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    def forward(self, x):
        raise NotImplementError()

In [11]:
# x = Variable(np.array(10))
# f = Function()
# y = f(x)

In [12]:
class Square(Function):
    def forward(self, x):
        return x **2

In [13]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# 3 함수 연결

In [14]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [15]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


# 4 수치 미분

In [16]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [17]:
f = Square()
x = Variable(np.array(3.0))
dy = numerical_diff(f, x)
print(dy)

6.000000000012662


In [18]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [19]:
4 * 0.5 * np.exp(2 * 0.5**2)

3.2974425414002564

# 6 수동 역전파

In [20]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [21]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()

In [22]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [23]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [24]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [25]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# 7 역전파 자동화

In [43]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [44]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

In [45]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [46]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [47]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [48]:
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [49]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# 8 재귀에서 반복문으로

In [62]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)    

In [72]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.)
y.backward()
print(x.grad)

3.297442541400256


In [57]:
print(y.creator)

# 9 함수를 더 편리하게  

In [76]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [77]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [79]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.)
y.backward()
print(x.grad)

3.297442541400256


In [80]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)    

In [81]:
x = np.array(1.)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [82]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [83]:
np.isscalar(np.float64(1.0))

True

In [84]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

# 10 테스트

In [87]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

In [89]:
SquareTest().test_forward()

In [90]:
!python -m unittest steps/tesp10.py

E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (errors=1)


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
